In [5]:
import torch_geometric.transforms as T
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ChebConv, SAGEConv
import matplotlib.pyplot as plt
import numpy as np
import utils
from torch_geometric.data import Data, Batch, DataLoader
from IPython.display import display, clear_output
import time

In [6]:
train_data = torch.load("data/MNISTSuperpixels_train_data.pt")
print(train_data[0])
test_data = torch.load("data/MNISTSuperpixels_test_data.pt")
print(test_data[0])

Data(edge_index=[2, 472], label=5, pos=[77, 2], x=[77, 3])
Data(edge_index=[2, 488], label=3, pos=[81, 2], x=[81, 3])


In [7]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

In [8]:
rotation_0 = T.RandomRotate(degrees=180, axis=0)
rotation_1 = T.RandomRotate(degrees=180, axis=1)
rotation_2 = T.RandomRotate(degrees=180, axis=2)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = ChebConv(in_channels, 32, K=2)
        self.conv2 = ChebConv(32, out_channels, K=2)
        
        self.conv1 = SAGEConv(in_channels, 32)
        self.conv2 = SAGEConv(32, out_channels)
        
#         self.conv1 = GCNConv(in_channels, 32)
#         self.conv2 = GCNConv(32, out_channels)
        
    def forward(self, x, edge_index):
#         x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
model = Net(dataset.num_node_features, dataset.num_classes)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def evaluate():
    acc = 0
    for data in test_loader:
        data = data.to(device)
        out = model()
    print(acc)

In [ ]:
model.train()

for i in range(5):
    
    for data in train_loader:
        data = data.to(device)
        
        optimizer.zero_grad()

        out = model()
        
        loss = F.nll_loss(out, data.label)
    
        print(loss)
        
        loss.backward()
        optimizer.step()
    
    evaluate()